# Neural Machine Translation using Seq2Seq Models

In HW06, we will be training our own machine translation system.

In [1]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 4.6 MB/s 


Mount your google drive here. We will be storing our training data in the google drive folder named `hw06_data`

In [33]:
from google.colab import drive
#drive._mount('/content/drive', force_remount=True)
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Make sure you are using a GPU. Go to Runtime -> Change Runtime Type -> GPU.

After running the below cell, if your notebook is configured to run on GPU, you should see a `device(type='cuda')` output

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchmetrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Object to store our language data. In this object, we assign a unique integer ID to every word in the vocab. We also assign new `<SOS>`, `<EOS>` and `<OOV>` tokens for start-of-sentence, end-of-sentence and out-of-vocab tokens respectively

In [4]:
SOS_token = 0
EOS_token = 1
OOV_token = 2


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "OOV"}
        self.n_words = 3  # Count SOS, EOS and OOV

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Standard text pre-processing and string normalization. We convert unicode to ASCII and remove all punctuations

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove punctuations
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = s.translate(str.maketrans('', '', string.punctuation))
    return s

In [6]:
def readLangs(lang1, lang2, path_to_corpus):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(path_to_corpus, encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('|||')] for l in lines]
        
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

We only consider sentences that are max 10 words in length. Think about why we might need to restrict the maximum number of words in a sentence.

In [7]:
MAX_LENGTH = 10


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Read in the data. Ensure that the path to the data files are correct

In [8]:
def prepareData(lang1, lang2, path_to_corpus):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, path_to_corpus)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs) # limitation to max length 10
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0]) # english sentence
        output_lang.addSentence(pair[1]) # lang sentence
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


# Give your input path here
HI_PATH = "/content/gdrive/MyDrive/Colab Notebooks/hw06_data/eng_hin/ted-train.orig.eng-hin"
ZH_PATH = "/content/gdrive/MyDrive/Colab Notebooks/hw06_data/eng_zh/ted-train.orig.eng-zh"
input_lang_train_en_hi, output_lang_train_hi, train_pairs_hi = prepareData('en', 'hi', HI_PATH)
input_lang_train_en_zh, output_lang_train_zh, train_pairs_zh = prepareData('en', 'zh', ZH_PATH)
print(random.choice(train_pairs_hi))
print(random.choice(train_pairs_zh))

Reading lines...
Read 18798 sentence pairs
Trimmed to 4493 sentence pairs
Counting words...
Counted words:
en 4162
hi 4809
Reading lines...
Read 5534 sentence pairs
Trimmed to 2142 sentence pairs
Counting words...
Counted words:
en 2906
zh 2395
['that is a huge transformation', 'यह एक विशाल परिवरतन ह']
['does amazon carry it', '亚马逊？']


Design your encoder model here

In [15]:
# Set your hidden size here
HIDDEN_SIZE = 256

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        # Assign hidden_size
        self.hidden_size = hidden_size ##YOUR CODE HERE

        # Create nn.Embedding layer with (input_size, hidden_size)
        self.embedding = nn.Embedding(input_size, hidden_size) ##YOUR CODE HERE

        # Create nn.GRU layer with (hidden_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, 1)##YOUR CODE HERE

    def forward(self, input, hidden):
        # Run the input through the embedding layer
        embedded = self.embedding(input) ##YOUR CODE HERE: torch.Size([1, 256])

        # Reshape with (1, 1, -1)
        embedded = embedded.reshape(1, 1, -1) ##YOUR CODE HERE: torch.Size([1, 1, 256])


        # Run both the embedded and hidden through GRU
        output, hidden = self.gru(embedded, hidden) ##YOUR CODE HERE: (1, 1, HIDDEN_SIZE), (1, 1, HIDDEN_SIZE)

        # Return both output and hidden
        return output, hidden

    def initHidden(self):
        # Create a torch tensor of zeros of shape (1, 1, HIDDEN_SIZE)
        return torch.zeros(1,1,self.hidden_size).to(device) ##YOUR CODE HERE

dummy_in = torch.randint(1, 10, (1,), device=device) # tensor([7], device='cuda:0')
dummy_encoder = EncoderRNN(100, HIDDEN_SIZE).to(device) # 
assert dummy_encoder.initHidden().shape == (1, 1, HIDDEN_SIZE) # init hidden
dummy_out, dummy_hid = dummy_encoder.forward(dummy_in, dummy_encoder.initHidden())
assert dummy_out.shape == (1, 1, HIDDEN_SIZE)
assert dummy_hid.shape == (1, 1, HIDDEN_SIZE)

Design your decoder model here

In [16]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        # Assign hidden_size
        self.hidden_size = hidden_size ##YOUR CODE HERE

        # Create nn.Embedding layer with (output_size, hidden_size)
        self.embedding = nn.Embedding(output_size, hidden_size) ##YOUR CODE HERE

        # Create nn.GRU layer with (hidden_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, 1) ##YOUR CODE HERE

        # Create a nn.Linear layer with (hidden_size, output_size)
        self.out = nn.Linear(hidden_size, output_size) ##YOUR CODE HERE

        # Create a nn.LogSoftmax layer with (dim=1)
        self.softmax = nn.LogSoftmax(dim = 1) ##YOUR CODE HERE

    def forward(self, input, hidden):
        # Run the input through the embedding layer
        input = self.embedding(input) ##YOUR CODE HERE

        # Reshape the input with (1, 1, -1)
        input = input.reshape(1,1,-1) ##YOUR CODE HERE

        # Use relu activation 
        input = F.relu(input)

        # Run both the input and hidden through GRU
        output, hidden = self.gru(input, hidden) ##YOUR CODE HERE
        
        # Reshape the output with (1, -1)
        output = output.reshape(1,-1) ##YOUR CODE HERE

        # Run the output through the linear layer
        output = self.out(output) ##YOUR CODE HERE

        # Get softmax scores
        output = self.softmax(output) ##YOUR CODE HERE

        # Return both output and hidden
        return output, hidden

    def initHidden(self):
        # Create a torch tensor of zeros of shape (1, 1, HIDDEN_SIZE)
        return torch.zeros(1,1,self.hidden_size).to(device) ##YOUR CODE HERE

dummy_in = torch.randint(1, 10, (1,), device=device)
dummy_decoder = DecoderRNN(HIDDEN_SIZE, 100).to(device)
assert dummy_decoder.initHidden().shape == (1, 1, HIDDEN_SIZE)
dummy_out, dummy_hid = dummy_decoder.forward(dummy_in, dummy_decoder.initHidden())
assert dummy_out.shape == (1, 100)
assert dummy_hid.shape == (1, 1, HIDDEN_SIZE)

Helper functions to convert the sentences into vector. Given a sentence, we convert it into a vector of word IDs

In [17]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index.get(word, OOV_token) for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(input_lang, output_lang, pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


Implement the main training loop here. This function takes in one input tensor and one output tensor and does a forward pass, backward pass and weight updates. 

In [18]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, only_forward_pass=False):
    # Initialize the hidden layer for encoder
    encoder_hidden = encoder.initHidden() ##YOUR CODE HERE

    # Reset gradients for both encoder_optimizer and decoder_optimizer
    encoder_optimizer.zero_grad() ##YOUR CODE HERE
    decoder_optimizer.zero_grad() ##YOUR CODE HERE

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        # Run each input word through the encoder
        # You can access the current input as input_tensor[ei]
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden) ##YOUR CODE HERE)

    # First input to the decoder
    decoder_input = torch.tensor([[SOS_token]], device=device)

    # Assign the last encoder_hidden to decoder_hidden
    decoder_hidden = encoder_hidden ##YOUR CODE HERE

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            # Run decoder by providing decoder_input and decoder_hidden as input
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) ##YOUR CODE HERE)

            # Calculate loss
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            # Run decoder by providing decoder_input and decoder_hidden as input
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) ##YOUR CODE HERE)

            # Take the top output of current timestep of decoder. This will be input to next timestep
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    if not only_forward_pass:
        # Backprop by calling backward() function on loss
        loss.backward() ##YOUR CODE HERE

        # Update weights using step() on both encoder_optimizer and decoder_optimizer
        encoder_optimizer.step()##YOUR CODE HERE
        decoder_optimizer.step() ##YOUR CODE HERE

    return loss.item() / target_length

In [19]:
def trainIters(encoder, decoder, n_iters, input_lang, output_lang, pairs, learning_rate=0.01):
    # Initialize SGD optimizers for both encoder and decoder
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    # Convert words to tensors
    all_training_pairs = [tensorsFromPair(input_lang, output_lang, pair) for pair in pairs]

    # Create training and valid datasets
    random.shuffle(all_training_pairs)
    valid_pairs = all_training_pairs[:500]
    training_pairs = all_training_pairs[500:]
    
    # We will be using NLLLoss as criterion
    criterion = nn.NLLLoss()

    epoch_train_losses = []
    epoch_valid_losses = []

    # In each epoch, we go through all training examples
    for iter in range(1, n_iters + 1):

        # Train
        train_loss = 0.0
        for training_pair in training_pairs:
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                      decoder, encoder_optimizer, decoder_optimizer, criterion)
            train_loss += loss

        # Validate
        valid_loss = 0.0
        for val_pair in valid_pairs:
            input_tensor = val_pair[0]
            output_tensor = val_pair[1]
            loss = train(input_tensor, target_tensor, encoder,
                      decoder, encoder_optimizer, decoder_optimizer, criterion, only_forward_pass=True)
            valid_loss += loss

        avg_train_loss = train_loss / len(training_pairs)
        avg_valid_loss = valid_loss / len(valid_pairs)

        print("Epoch: {}/{}. Avg Train Loss: {}. Avg Valid Loss: {}".format(iter, n_iters, avg_train_loss, avg_valid_loss))

        epoch_train_losses.append(avg_train_loss)
        epoch_valid_losses.append(avg_valid_loss)

    return epoch_train_losses, epoch_valid_losses

Train the model. What do you notice about the training and validation losses?

In [20]:
# ENG - HIN
encoder_eng_hi = EncoderRNN(input_lang_train_en_hi.n_words, HIDDEN_SIZE).to(device)
decoder_eng_hi = DecoderRNN(HIDDEN_SIZE, output_lang_train_hi.n_words).to(device)

avg_train_losses_hi, avg_valid_losses_hi = trainIters(encoder_eng_hi, decoder_eng_hi, 30, input_lang_train_en_hi, output_lang_train_hi, train_pairs_hi)

# ENG - ZH
encoder_eng_zh = EncoderRNN(input_lang_train_en_zh.n_words, HIDDEN_SIZE).to(device)
decoder_eng_zh = DecoderRNN(HIDDEN_SIZE, output_lang_train_zh.n_words).to(device)

avg_train_losses_zh, avg_valid_losses_zh = trainIters(encoder_eng_zh, decoder_eng_zh, 30, input_lang_train_en_zh, output_lang_train_zh, train_pairs_zh)

Epoch: 1/30. Avg Train Loss: 4.8587927698139115. Avg Valid Loss: 3.7179735960279188
Epoch: 2/30. Avg Train Loss: 4.64201456489197. Avg Valid Loss: 4.016932969229561
Epoch: 3/30. Avg Train Loss: 4.390332147195442. Avg Valid Loss: 4.432969999313357
Epoch: 4/30. Avg Train Loss: 4.105104902559589. Avg Valid Loss: 4.698872936793735
Epoch: 5/30. Avg Train Loss: 3.7589293600371527. Avg Valid Loss: 4.988428626741683
Epoch: 6/30. Avg Train Loss: 3.389894002306302. Avg Valid Loss: 5.469907428332741
Epoch: 7/30. Avg Train Loss: 3.0211862115990535. Avg Valid Loss: 5.56451514993395
Epoch: 8/30. Avg Train Loss: 2.6542840060736843. Avg Valid Loss: 5.63467682729449
Epoch: 9/30. Avg Train Loss: 2.3218260580239214. Avg Valid Loss: 6.184317752429413
Epoch: 10/30. Avg Train Loss: 1.9795294545420634. Avg Valid Loss: 6.225307368006029
Epoch: 11/30. Avg Train Loss: 1.6640282825671135. Avg Valid Loss: 6.332162562234056
Epoch: 12/30. Avg Train Loss: 1.4003537360370304. Avg Valid Loss: 6.780699701036727
Epoch: 

In [21]:
# # ENG - HIN
# encoder_eng_hi = EncoderRNN(input_lang_train_en_hi.n_words, HIDDEN_SIZE).to(device)
# decoder_eng_hi = DecoderRNN(HIDDEN_SIZE, output_lang_train_hi.n_words).to(device)

# avg_train_losses_hi, avg_valid_losses_hi = trainIters(encoder_eng_hi, decoder_eng_hi, 30, input_lang_train_en_hi, output_lang_train_hi, train_pairs_hi)

# # ENG - ZH
# encoder_eng_zh = EncoderRNN(input_lang_train_en_zh.n_words, HIDDEN_SIZE).to(device)
# decoder_eng_zh = DecoderRNN(HIDDEN_SIZE, output_lang_train_zh.n_words).to(device)

# avg_train_losses_zh, avg_valid_losses_zh = trainIters(encoder_eng_zh, decoder_eng_zh, 30, input_lang_train_en_zh, output_lang_train_zh, train_pairs_zh)

In [30]:
!pwd

/content


In [31]:
with open("/content/gdrive/MyDrive/Colab Notebooks/losses_hi.txt", "w") as fp:
    for i,j in zip(avg_train_losses_hi, avg_valid_losses_hi):
      fp.write("{:.12f} {:.12f}\n".format(i, j))

with open("/content/gdrive/MyDrive/Colab Notebooks/losses_zh.txt", "w") as fp:
    for i,j in zip(avg_train_losses_zh, avg_valid_losses_zh):
      fp.write("{:.12f} {:.12f}\n".format(i, j))

Function to perform inference. Given an input sentence, this function returns the translated sentence

In [27]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(MAX_LENGTH):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

Try out a random input. 

1. Does the output make sense if given an input from the training set?
2. Does the output make sense if given an arbitrary input?

In [24]:
print(evaluate(encoder_eng_hi, decoder_eng_hi, "we can go into the water", input_lang_train_en_hi, output_lang_train_hi))

print(evaluate(encoder_eng_zh, decoder_eng_zh, "Five new flagship stores opened.", input_lang_train_en_zh, output_lang_train_zh))

['पानी', 'पानी', 'को', 'पानी', 'जा', 'सकत', 'ह।', '<EOS>']
['我意思係話，癌症點睇', '<EOS>']


Calculate CHRF score

In [25]:
hyp_hi = []
ref_hi = []

for sample in train_pairs_hi:
    x = sample[0]
    y_true = sample[1]
    y_pred_tokens = evaluate(encoder_eng_hi, decoder_eng_hi, x, input_lang_train_en_hi, output_lang_train_hi)

    if "<EOS>" in y_pred_tokens:
        y_pred_tokens.remove("<EOS>")
    y_pred = " ".join(y_pred_tokens)

    hyp_hi.append(y_pred)
    ref_hi.append([y_true])

hyp_zh = []
ref_zh = []

for sample in train_pairs_zh:
    x = sample[0]
    y_true = sample[1]
    y_pred_tokens = evaluate(encoder_eng_zh, decoder_eng_zh, x, input_lang_train_en_zh, output_lang_train_zh)

    if "<EOS>" in y_pred_tokens:
        y_pred_tokens.remove("<EOS>")
    y_pred = " ".join(y_pred_tokens)

    hyp_zh.append(y_pred)
    ref_zh.append([y_true])

metric = torchmetrics.CHRFScore()
print("Hindi CHRF Score", metric(hyp_hi, ref_hi))
print("Chinese CHRF Score", metric(hyp_zh, ref_zh))

/usr/local/lib/python3.7/dist-packages/torchmetrics/functional/text/chrf.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  total_n_grams[n] = tensor(sum(n_grams_counts[n].values()))
/usr/local/lib/python3.7/dist-packages/torchmetrics/functional/text/chrf.py:211: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  for n_gram in hyp_n_grams_counts[n]


Hindi CHRF Score tensor(0.8743)
Chinese CHRF Score tensor(0.8134)


In [32]:
with open("/content/gdrive/MyDrive/Colab Notebooks/predictions_hi.txt", "w") as fp:
    for i,j in zip(hyp_hi, ref_hi):
        fp.write("{} {}\n".format(i, str(j)))

with open("/content/gdrive/MyDrive/Colab Notebooks/predictions_zh.txt", "w") as fp:
    for i,j in zip(hyp_zh, ref_zh):
        fp.write("{} {}\n".format(i, str(j)))